## 1. 1번 데이터셋 전처리 코드

In [7]:
# iplot 라이브러리
import chart_studio
chart_studio.tools.set_credentials_file()
import chart_studio.plotly as py
import plotly.express as px
from chart_studio.plotly import plot, iplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import pandas as pd
import numpy as np

from datetime import * 
import seaborn as sns
%matplotlib inline

import re
import urllib.request

# 한글 폰트 설정
import matplotlib.pyplot as plt 
import matplotlib as mpl
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (14,4)

mpl.rcParams['axes.unicode_minus'] = False

In [34]:
# 경로 자기 파일이랑 맞게 바꿔주자. 
# data_2018 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/1. 2018년 항공 스케줄 데이터(API).xlsx')
# data_2019 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/1. 2019년 항공 스케줄 데이터(API).xlsx')
# data_2020 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/1. 2020년 항공 스케줄 데이터(API).xlsx')
# data_2021 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/1. 2021년 항공 스케줄 데이터(API).xlsx')
# data_2022 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/1. 2022년 항공 스케줄 데이터(API).xlsx')


# # # 김해 - 김포
# data_2018 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/김해/1. 2018년 항공 스케줄 데이터(API).xlsx')
# data_2019 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/김해/1. 2019년 항공 스케줄 데이터(API).xlsx')
# data_2020 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/김해/1. 2020년 항공 스케줄 데이터(API).xlsx')
# data_2021 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/김해/1. 2021년 항공 스케줄 데이터(API).xlsx')
# data_2022 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/김해/1. 2022년 항공 스케줄 데이터(API).xlsx')


# # 김해 - 제주
data_2018 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/제주-김해/1. 2018년 항공 스케줄 데이터(API).xlsx')
data_2019 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/제주-김해/1. 2019년 항공 스케줄 데이터(API).xlsx')
data_2020 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/제주-김해/1. 2020년 항공 스케줄 데이터(API).xlsx')
data_2021 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/제주-김해/1. 2021년 항공 스케줄 데이터(API).xlsx')
data_2022 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/1번/제주-김해/1. 2022년 항공 스케줄 데이터(API).xlsx')



# 공공데이터 스케줄 (예진님 준거)
matching_data_21 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/스케줄 공공데이터/한국공항공사_국내선항공기스케줄_20210910.xlsx')
matching_data_22 = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/스케줄 공공데이터/한국공항공사_국내선 항공기스케줄_20220919.xlsx')


# 항공 기종 매칭 데이터
matching_korea_airplane = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/기종 데이터/9. 기종별 매칭.xlsx')
# 항공사별 항공기 정보 
airplane_info = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/기종 데이터/7. 항공사별 항공기 정보.xlsx')

# 조인 시켜주기 위해 ICAO 열을 일단 기종으로 변경 
airplane_info.rename({'ICAO':'기종'}, axis=1, inplace=True)


# 기종 매칭 
data_2021 = pd.merge(data_2021, matching_data_21, how='left',on='운항편명')
data_2022 = pd.merge(data_2022, matching_data_22, how='left',on='운항편명')

# 데이터 없는 나머지는 용량이 많은 2021로 조인해주자 
data_2018 = pd.merge(data_2018, matching_data_21, how='left',on='운항편명')
data_2019 = pd.merge(data_2019, matching_data_21, how='left',on='운항편명')
data_2020 = pd.merge(data_2020, matching_data_21, how='left',on='운항편명')

In [35]:
def process(data, year):
    data.drop(data[data['항공사명']=='하이에어'].index, inplace=True)
    data.drop(data[data['항공사명']=='에어프레미아'].index, inplace=True)
    data.drop(data[data['항공사명']=='에어필립'].index, inplace=True)
    
    # NaN 전부 보내주자
    data.dropna(inplace=True)
    
    # 인덱스 초기화
    data.reset_index(drop=True, inplace=True)


    
    
    # 애매한 친구들 바꿔주자
    data['기종'] = data['기종'].apply(lambda x: 'BCS3' if x == 'A220' else x)
    data['기종'] = data['기종'].apply(lambda x: 'B763' if x == 'B767' else x)
    
    # 제주공항 -> CJU ,김포공항 -> GMP
#     data['출발공항'] = data['출발공항'].apply(lambda x: 'GMP' if x == '서울/김포' else 'CJU')
#     data['도착공항'] = data['출발공항'].apply(lambda x: 'GMP' if x == '서울/김포' else 'CJU')
                 
                 
#     # 운항시간 계산 전에 시간을 int형으로 변환
#     data['출발예정시간'] = data['출발예정시간'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1]))
#     data['도착예정시간'] = data['도착예정시간'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1]))
    
     # 실제 운항시간 계산
    data['운항시간'] = data['도착예정시간']//100*60 + data['도착예정시간']%100 - data['출발예정시간']//100*60 - data['출발예정시간']%100
    
    # 기종명 매칭 
    code_to_model = dict([(key,value) for key, value in zip(matching_korea_airplane['기종 1'], matching_korea_airplane['기종 2'])])
    data['기종명'] = data['기종'].map(lambda x:code_to_model[x])
    
    
    # 항공사명 -> 항공사로 열 이름 변경
    data.rename({'항공사명':'항공사'}, axis=1, inplace=True)
    
    data = pd.merge(data, airplane_info, how='left', on=['항공사','기종명']).drop(['기종_x', '기종_y'], axis=1)
    
    
    # 여객수 열 추가(연도별 항공사별 항공통계 -> 여객(명)/공급(석) 수치를 구해서 우리 데이터에 항공사 별로 곱해주자)
    if year == 2022:
        ratio = {'대한항공':0.84861504, '아시아나항공':0.835902988, '에어부산':0.86152892, 
                     '에어서울' : 0.93397103, '제주항공': 0.918230233, '진에어':0.834867913, '티웨이항공':0.88084639}
    elif year == 2021:
        ratio = {'대한항공':0.769867166, '아시아나항공':0.784258806, '에어부산':0.795203654, 
                     '에어서울' : 0.926177813, '제주항공': 0.888309397, '진에어':0.80295672, '티웨이항공':0.880846396}
    elif year == 2020:
        ratio = {'대한항공':0.727634146, '아시아나항공':0.717727166, '에어부산':0.742723332, 
                     '에어서울' : 0.876381662, '제주항공': 0.843597915, '진에어':0.765062833, '티웨이항공':0.82352292}
    elif year == 2019:
        ratio = {'대한항공':0.831612, '아시아나항공':0.887486, '에어부산':0.847954, 
                 '에어서울' : 0.903508, '제주항공': 0.950423, '진에어':0.936403, '티웨이항공':0.942232}              
                      
    elif year == 2018:
        ratio = {'대한항공':0.754804, '아시아나항공':0.861673, '에어부산':0.870346, 
                     '제주항공': 0.948732, '진에어':0.942583, '티웨이항공':0.943396}
                  
    
    data['전체여객 비율'] = data['항공사'].apply(lambda x: ratio[x])

    data['전체여객'] = data['전체여객 비율'] * data['좌석수'] 
    data['전체여객'] = data['전체여객'].astype('int')

    data.drop('전체여객 비율', axis=1, inplace=True)
    
    
    
    
    data.rename({'출발예정시간':'출발시간','도착예정시간':'도착시간', '기종명':'기종'}, axis=1, inplace=True)
    
    data.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    

    
    return data

In [36]:
# 함수 실행
data_2018 = process(data_2018, 2018)
data_2019 = process(data_2019, 2019)
data_2020 = process(data_2020, 2020)
data_2021 = process(data_2021, 2021)
data_2022 = process(data_2022, 2022)

## 3장. 온실가스 배출 현황

In [37]:
def calculate_emmission(data):
    # 운항거리(제주-김포노선 449km)
#     data['운항거리'] = 449
    
    # 운항거리(부산-김포 노선 327km)
#     data['운항거리'] = 327
    
        
#     # 운항거리(부산-제주 노선 291km)
    data['운항거리'] = 291
    
    
    # 배출계수 데이터 
    dispose_data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:2/Movie_Project/최종 항공 데이터/기종 데이터/10. 기종별 배출계수.xlsx')
    
    # 기종별 배출계수 병합
    dispose_data.drop(columns=['기종 3'], axis=1, inplace=True)
    dispose_data['fuel burn'] = dispose_data['fuel burn'].map(lambda x:float(x.split( )[0]))
    
    
    df_calculated = data.merge(dispose_data, how='left', left_on='기종', right_on = '기종 2').drop(columns=['기종 2'])
    
   # 온실가스 계산 : LTO 단계 배출량 + 순항 단계 배출량
    df_calculated['cruise_fuel(kg)'] = df_calculated['운항거리'] * df_calculated['fuel burn']
    df_calculated['total_fuel(kg)'] = df_calculated['cruise_fuel(kg)'] + df_calculated['LTO_fuel(kg/LTO)']
    # CO2 등가계수(CO2:1, CH4:21, N2O:310)
    df_calculated['CO2(kgCO2eq)'] = df_calculated['LTO_CO2(kg/LTO)'] + df_calculated['cruise_fuel(kg)'] * df_calculated['EF_Cruise_CO2(kg/t of fuel)']/1000 
    df_calculated['CH4(kgCO2eq)'] = df_calculated['LTO_CH4(kg/LTO)'] + df_calculated['cruise_fuel(kg)'] * df_calculated['EF_Cruise_CH4(kg/t of fuel)']/1000 * 21
    df_calculated['N2O(kgCO2eq)'] = df_calculated['LTO_N2O(kg/LTO)'] + df_calculated['cruise_fuel(kg)'] * df_calculated['EF_Cruise_N2O(kg/t of fuel)']/1000 * 310
    df_calculated['total_emmission(kgCO2eq)'] =  df_calculated['CO2(kgCO2eq)'] + df_calculated['CH4(kgCO2eq)'] + df_calculated['N2O(kgCO2eq)']

    df_calculated = df_calculated.drop(columns=['fuel burn','LTO_CO2(kg/LTO)','LTO_CH4(kg/LTO)','LTO_N2O(kg/LTO)','EF_Cruise_CO2(kg/t of fuel)', 'EF_Cruise_CH4(kg/t of fuel)','EF_Cruise_N2O(kg/t of fuel)'])
    
    return df_calculated

In [38]:
# 함수 실행 
emmission_data_2018 = calculate_emmission(data_2018)
emmission_data_2019 = calculate_emmission(data_2019)
emmission_data_2020 = calculate_emmission(data_2020)
emmission_data_2021 = calculate_emmission(data_2021)
emmission_data_2022 = calculate_emmission(data_2022)

In [39]:
# 계산된 기종별 김포-제주 노선 운항편당 온실가스 배출량 
total_emmission_data = pd.concat([emmission_data_2018, emmission_data_2019, emmission_data_2020, emmission_data_2021, emmission_data_2022])

total_emmission_data[['기종','total_emmission(kgCO2eq)']].drop_duplicates().sort_values('total_emmission(kgCO2eq)')

,기종,total_emmission(kgCO2eq)
9,B737-8,5288.73841
18,A220-300,5418.33235
16,B737-700,5431.59391
129,B737-900,5473.87261
0,B737-800,5973.73495
2,A321-neo,6074.95430
1,A321-200,6361.91231


### 3-2. 온실가스 현황 분석

In [40]:
# (1) 연도별 항공 온실가스 구성(평균) 기체비율(CO2, CH4, N2O)

def gas_average(data):
    co2_average = data['CO2(kgCO2eq)'].mean()
    ch4_average = data['CH4(kgCO2eq)'].mean()
    n2O_average = data['N2O(kgCO2eq)'].mean()
    
    gas_average_data = pd.DataFrame({'기체':['CO2', 'CH4', 'N2O'],
   '항공기 배출량':[co2_average, ch4_average, n2O_average]
    })
    
    fig=px.pie(gas_average_data, values='항공기 배출량',names='기체',color='기체',color_discrete_map={'CO2':'lightpink','N2O':'lightskyblue','CH4':'lightgrey'})
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(title='<b>항공 온실가스 구성 기체비율<b>',title_x=0.47,width=600,height=400)
    fig.show()
    
    return gas_average_data

In [41]:
gas_average(emmission_data_2018)

,기체,항공기 배출량
0,CO2,6013.145783
1,CH4,0.105652
2,N2O,31.014575


In [42]:
gas_average(emmission_data_2019)

,기체,항공기 배출량
0,CO2,6017.121656
1,CH4,0.104220
2,N2O,31.083411


In [43]:
gas_average(emmission_data_2020)

,기체,항공기 배출량
0,CO2,5904.406840
1,CH4,0.100542
2,N2O,30.416538


In [44]:
gas_average(emmission_data_2021)

,기체,항공기 배출량
0,CO2,5993.607810
1,CH4,0.103591
2,N2O,30.931523


In [45]:
gas_average(emmission_data_2022)

,기체,항공기 배출량
0,CO2,5758.123370
1,CH4,0.103598
2,N2O,28.818863


In [46]:
total_emmission_data['연도'] = total_emmission_data['운항일자'].str[0:4]

In [47]:
# (2) 연도별 항공기 온실가스 총 배출량과 운항횟수
def year_emmission_and_aircount(total_data):
    total_data['연도'] = total_data['운항일자'].str[0:4]
    data1=total_data.groupby('연도').sum().loc[:,'total_emmission(kgCO2eq)'].reset_index()
    data2=total_data.groupby('연도').size().reset_index().rename(columns={0:'운항횟수'})
    data=pd.merge(data1,data2,how='inner',on='연도')

    fig = make_subplots(specs=[[{'secondary_y': True}]])
    fig.add_trace(go.Bar(x=data['연도'],y=data['운항횟수'],name='운항횟수',marker=dict(color=['lightgrey', 'lightpink', 'lightskyblue', 'lightgreen', 'lightskyblue'])),secondary_y=False,)
    fig.add_trace(go.Scatter(x=data['연도'],y=data['total_emmission(kgCO2eq)'],name='총 온실가스 배출량'),secondary_y=True,)
    fig.update_layout(title='<b>연도별 김포-제주 노선 운항시 항공기 온실가스 총 배출량과 운항횟수<b>',title_x=0.47,width=800,height=400,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))
    fig.update_xaxes(title_text='연도')
    fig.update_yaxes(tickformat='%d',title_text='<b>운항횟수(회)<b>',title_font=dict(size=12,family='Courier', color='crimson'), secondary_y=False)
    fig.update_yaxes(tickformat='%d',title_text='<b>총 온실가스 배출량(kgCO2eq)<b>',title_font=dict(size=12,family='Courier', color='crimson'), secondary_y=True)
    fig.show()
    
    

In [48]:
year_emmission_and_aircount(total_emmission_data)

In [49]:
# (3) 2018-2022년 월별 김포-제주 노선 항공 온실가스 배출량
# color_discrete_map={'01':'lightgrey','04':'lightpink','07':'lightgreen','10':'lightskyblue'}
def month_emmission(total_data):
    total_data['월'] = total_data['운항일자'].str[5:7]
    data = total_data.groupby(['연도','월']).sum().loc[:,'total_emmission(kgCO2eq)'].reset_index()
    fig=px.bar(data,x='연도',y='total_emmission(kgCO2eq)',barmode='group',color='월')
    fig.update_layout(title_text='<b>연도별 월별 김포-제주 노선 항공 온실가스 배출량<b>',title_x=0.5,width=800,height=400)
    fig.show()   

In [50]:
month_emmission(total_emmission_data)

In [51]:
# (4-1) 연도별 항공사별 온실가스 배출량 총합
def month_emmission(year_data):

    #항공사별 색깔 딕셔너리
    airportcolor={'아시아나항공':'lightpink','대한항공':'lightskyblue','제주항공':'thistle','진에어':'navajowhite','티웨이항공':'lightgreen','에어부산':'lightsalmon','에어서울':'royalblue'}

    data=year_data.groupby(['항공사']).sum().loc[:,'total_emmission(kgCO2eq)'].reset_index()
    fig=px.pie(data,values='total_emmission(kgCO2eq)',names='항공사',color='항공사',color_discrete_map=airportcolor)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(title_text='<b>연도별 김포-제주 노선 운항시 항공기 온실가스 총 배출량<b>',title_x=0.5,legend=dict(yanchor="bottom",y=0.6,xanchor="right",x=0.9))
    fig.show()

In [52]:
month_emmission(emmission_data_2022)

In [53]:
# (4-2) 대한항공, 아시아나항공의 1인당 평균 항공 온실가스 배출량 2018-2022년 추이(다른 항공사도 넣어줄까)

def airline_emmission(total_data):
    data = total_data.groupby(['연도','항공사']).sum().loc[:,{'total_emmission(kgCO2eq)','좌석수'}].reset_index()
    data['total_emmission(kgCO2eq)/person']=data['total_emmission(kgCO2eq)']/data['좌석수']
    fig=go.Figure()
    data1=data[data['항공사']=='대한항공']
    data2=data[data['항공사']=='아시아나항공']
    data3=data[data['항공사']=='진에어']
    data4=data[data['항공사']=='에어부산']
    data5=data[data['항공사']=='제주항공']
    data6=data[data['항공사']=='에어서울']
    data7=data[data['항공사']=='티웨이항공']
    
    
    fig.add_trace(go.Scatter(x=data1['연도'],y=data1['total_emmission(kgCO2eq)/person'],name='대한항공'))
    fig.add_trace(go.Scatter(x=data2['연도'],y=data2['total_emmission(kgCO2eq)/person'],name='아시아나항공'))
    fig.add_trace(go.Scatter(x=data3['연도'],y=data3['total_emmission(kgCO2eq)/person'],name='진에어'))
    fig.add_trace(go.Scatter(x=data4['연도'],y=data4['total_emmission(kgCO2eq)/person'],name='에어부산'))
    fig.add_trace(go.Scatter(x=data5['연도'],y=data5['total_emmission(kgCO2eq)/person'],name='제주항공'))
    fig.add_trace(go.Scatter(x=data6['연도'],y=data6['total_emmission(kgCO2eq)/person'],name='에어서울'))
    fig.add_trace(go.Scatter(x=data7['연도'],y=data7['total_emmission(kgCO2eq)/person'],name='티웨이항공'))
        
    fig.update_layout(title='<b>대한항공과 아시아나항공의 1인당 온실가스 배출량의 연도별 추이<b>',title_x=0.47,width=800,height=400)
    fig.update_xaxes(title_text='연도')
    fig.update_yaxes(title_text='1인당 온실가스 배출량')
    fig.show() 
    
    

In [54]:
airline_emmission(total_emmission_data)

In [55]:
## EDA 용 데이터 전처리

def preprocessing_eda(df):
    # 추가 변수 생성
    df['출발시각'] = df['출발시간'] // 100
    df['도착시각'] = df['도착시간'] // 100
    
    
    
    df['월'] = pd.to_numeric(df['운항일자'].str[5:7])
    df['운항일자'] = pd.to_datetime(df['운항일자'])
    df['요일'] = df['운항일자'].dt.weekday # 월:0, 화:1, 수:2, 목:3, 금:4, 토:5, 일:6
    df['운항일자'] = df['운항일자'].map(lambda x:str(x)[:10]) # 다시 string으로 변환
    df['기종(좌석수)'] = df['기종'] + '(' + df['좌석수'].astype(str) + ')'
    new_columns = df.columns[-1:].to_list() + df.columns[:-1].to_list()
    df = df[new_columns]
    
    return df

In [56]:
total_eda_data = preprocessing_eda(total_emmission_data)

In [59]:
total_eda_data.to_excel('eda용 최종 데이터(제주-부산).xlsx')